In [1]:
import pandas as pd
import numpy as np
from io import StringIO


df = pd.read_csv('./data/Processed_Data_Lowen_Basketball/full_athlete_metrics.csv')
df['Date'] = pd.to_datetime(df['Date'])

# ---------------------------------------------------------
# 2. CALCULATE FATIGUE (The "Decoupling" Metric)
# ---------------------------------------------------------
# We want to know: Is the RPE (Subjective) much higher than the Heart Rate (Objective)?
# High Decoupling = Player feels harder than their body is actually working (CNS Fatigue / Stress)

# Avoid division by zero
df['LoadFC'] = df['LoadFC'].replace(0, np.nan) 

# Calculate simple ratio (RPE Load per unit of Heart Rate Load)
df['Physio_Cost_Index'] = df['LoadRPE'] / df['LoadFC']

# Z-Score: Is today's ratio weird for THIS player?
df['Fatigue_Z_Score'] = df.groupby('Player')['Physio_Cost_Index'].transform(
    lambda x: (x - x.mean()) / x.std()
)

# Logic: If Ratio spikes (Z > 1.5), they feel terrible but body is fine (Mental/CNS Fatigue)
df['Status_Fatigue'] = np.where(df['Fatigue_Z_Score'] > 1.5, 'HIGH CNS FATIGUE', 'OK')


# ---------------------------------------------------------
# 3. IMPUTE EXTERNAL DATA (Wellness + Schedule)
# ---------------------------------------------------------
np.random.seed(42)

# A. Wellness (Sleep/Soreness)
df['Wellness_Sleep'] = np.random.randint(1, 6, size=len(df)) # 1-5 Scale
df['Wellness_Soreness'] = np.random.randint(1, 6, size=len(df))

# B. Schedule (Days until next game)
# Let's assume a game is 3 days away from the last date
max_date = df['Date'].max()
game_date = max_date + pd.Timedelta(days=3)
df['Days_Until_Game'] = (game_date - df['Date']).dt.days


# ---------------------------------------------------------
# 4. FINAL DASHBOARD FLAGS (Business Logic)
# ---------------------------------------------------------

def get_dashboard_status(row):
    reasons = []
    status = "GREEN"
    
    # Rule 1: Injury Risk (AC Ratio)
    if row['AC_Ratio'] > 1.5:
        status = "RED"
        reasons.append("High Injury Risk (AC > 1.5)")
    elif row['AC_Ratio'] < 0.8 and row['AC_Ratio'] > 0:
        status = "YELLOW"
        reasons.append("Undertraining")

    # Rule 2: Wellness Override
    if row['Wellness_Sleep'] <= 2:
        status = "RED" if status != "RED" else "RED"
        reasons.append("Poor Sleep")

    # Rule 3: Tapering Violation
    if row['Days_Until_Game'] <= 2 and row['LoadRPE'] > 600:
        status = "RED"
        reasons.append("Training too hard before game")
        
    return pd.Series([status, ", ".join(reasons)])

df[['Dashboard_Color', 'Dashboard_Insight']] = df.apply(get_dashboard_status, axis=1)

# ---------------------------------------------------------
# 5. PREVIEW
# ---------------------------------------------------------
# Show relevant columns for the dashboard
output_cols = ['Date', 'Player', 'LoadRPE', 'LoadFC', 'AC_Ratio', 'Dashboard_Color', 'Dashboard_Insight']
print(df[output_cols].head())

        Date   Player  LoadRPE  LoadFC  AC_Ratio Dashboard_Color  \
0 2023-08-07    ARNAS      0.0     NaN       0.0           GREEN   
1 2023-08-07    BANGO      0.0     NaN       0.0           GREEN   
2 2023-08-07    BARRA      0.0     NaN       0.0           GREEN   
3 2023-08-07  BRANDON    135.0     NaN       1.0           GREEN   
4 2023-08-07    FERDI    135.0     NaN       1.0           GREEN   

  Dashboard_Insight  
0                    
1                    
2                    
3                    
4                    
